In [1]:
import numpy as np
from numpy import concatenate
import pandas as pd
from pandas import read_csv, concat, DataFrame
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from ipywidgets import interact_manual, interact
# Set the Seaborn context to 'talk' and style to 'whitegrid'
sns.set_context('talk')
sns.set_style('white')


In [2]:
# Read Excel file
excel_file = 'weeklydata.xlsx'
dataset = pd.read_excel(excel_file, engine='openpyxl')

In [3]:
# View data and column headings
print("Excel Data:")
print(dataset.head())
print("\nColumn Headings:")
print(dataset.columns)

Excel Data:
    state  year   datetime  epiweek  cases  tempmaxw  tempminw  tempw  \
0  Bauchi  2018 2018-01-07        1    1.0      29.5      10.8   19.0   
1  Bauchi  2018 2018-01-14        2    0.0      27.6      10.8   19.3   
2  Bauchi  2018 2018-01-21        3    0.0      30.5      11.7   21.2   
3  Bauchi  2018 2018-01-28        4    0.0      28.5      13.1   20.4   
4  Bauchi  2018 2018-02-04        5    0.0      34.6      15.6   24.8   

   humidityw  precipw  precipcovw  
0       36.1      0.0         0.0  
1       28.1      0.0         0.0  
2       24.8      0.0         0.0  
3       30.9      0.0         0.0  
4       23.8      0.0         0.0  

Column Headings:
Index(['state', 'year', 'datetime', 'epiweek', 'cases', 'tempmaxw', 'tempminw',
       'tempw', 'humidityw', 'precipw', 'precipcovw'],
      dtype='object')


In [4]:
from ipywidgets import interact_manual
@interact(state=["Bauchi", "Edo", "Ondo"], year = [y for y in range(2018, 2024)])
def viz(state, year):    
    # Heat Map showing the correlation between all variables including the target
    ds_state = dataset[(dataset['state'] == state) & (dataset['year'] == year)].copy()
    ds_state.set_index('datetime', inplace=True)
    ds_state.drop(['year', 'epiweek','state'], axis=1, inplace=True)    
    corr=ds_state.corr(method='spearman')    
    fig, ax = plt.subplots(figsize=(8,8))
    matrix = np.triu(corr) # Getting the lower traingle of the correlation matrix
    cbar_kws={"label": "Correlation", "shrink":1}
    heatmap=sns.heatmap(data=corr, linewidths=1, square=False, cmap='Reds', ax=ax, annot=True,annot_kws={"size": 10}, mask=matrix, fmt= ".2f",cbar_kws=cbar_kws)
    fig.suptitle('Heatmap of Correlation Between Data Features for {} state'.format(state), fontsize=18, y=.94, x=.43)
    plt.tight_layout()
    plt.savefig('{}_heatmap.svg'.format(state), bbox_inches='tight')



interactive(children=(Dropdown(description='state', options=('Bauchi', 'Edo', 'Ondo'), value='Bauchi'), Dropdo…

In [6]:
from ipywidgets import interact_manual
@interact(state=["Bauchi", "Edo", "Ondo"], )
def viz(state):
    year = [y for y in range(2018, 2024, 1)]
    # Heat Map showing the correlation between all variables including the target
    yearly_corr = []
    for y in year:
        y_cases_corr = []
        ds_state = dataset[(dataset['state'] == state) & (dataset['year'] == y)].copy()
        ds_state.set_index('datetime', inplace=True)
        ds_state.drop(['year', 'epiweek','state'], axis=1, inplace=True)    
        corr=ds_state.corr(method='spearman')
        # y_cases_corr.append(np.abs(corr['cases']['tempmaxw']))
        # y_cases_corr.append(np.abs(corr['cases']['tempminw']))
        # y_cases_corr.append(np.abs(corr['cases']['tempw']))
        # y_cases_corr.append(np.abs(corr['cases']['humidityw']))
        # y_cases_corr.append(np.abs(corr['cases']['precipw']))
        # y_cases_corr.append(np.abs(corr['cases']['precipcovw']))
        y_cases_corr.append(corr['cases']['tempmaxw'])
        y_cases_corr.append(corr['cases']['tempminw'])
        y_cases_corr.append(corr['cases']['tempw'])
        y_cases_corr.append(corr['cases']['humidityw'])
        y_cases_corr.append(corr['cases']['precipw'])
        y_cases_corr.append(corr['cases']['precipcovw'])

        yearly_corr.append(y_cases_corr)
    yearly_corr =np.array(yearly_corr)
    
    fig = plt.figure(figsize=(18, 8))
    plt.plot(year, yearly_corr.T[0], label='tempmaxw', color ='red', linestyle ='dotted')
    plt.plot(year, yearly_corr.T[1], label='tempminw', color ='green', linestyle ='dashed')
    plt.plot(year, yearly_corr.T[2], label='tempw',  color ='blue', linestyle ='dashdot')
    plt.plot(year, yearly_corr.T[3], label='humidityw', color ='black', linestyle ='solid')
    plt.plot(year, yearly_corr.T[4], label='precipw', color ='orange', linestyle =(0, (3, 10, 1, 10, 1, 10)))
    plt.plot(year, yearly_corr.T[5], label='precipcovw', color ='purple', linestyle =(0, (3, 5, 1, 5, 1, 5)))
    
    plt.ylabel("Cases count correlation")
    plt.xticks(np.arange(min(year), max(year)+1, 1.0))
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.tight_layout()
    plt.title("Spearman's cases absolute correlation value plot for {} state".format(state))
    plt.tight_layout()
    plt.savefig('{}_correlation.svg'.format(state), bbox_inches='tight')





interactive(children=(Dropdown(description='state', options=('Bauchi', 'Edo', 'Ondo'), value='Bauchi'), Output…